In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


In [ ]:
root_path = './dataset/'

In [ ]:
comportamiento_df = pd.read_csv(f'{root_path}comportamiento_tarjetasvisa.csv', sep = ',')

In [ ]:
informacion_df = pd.read_csv(f'{root_path}informacion_adicional_tarjetas.csv', sep = ',')

In [ ]:
comportamiento_df['ID_CLIENTE'].value_counts().shape[0]

### Definición de mal pagador

In [ ]:
comportamiento_df.drop_duplicates()
comportamiento_df

In [ ]:
comportamiento_df.isnull().sum()

##### 20% de las observaciones son nulas en columna DIAS_VENCIDOS

In [ ]:
grupo_df = comportamiento_df.groupby(["ID_CLIENTE","MESES"]).max()
grupo_df

In [ ]:
filtro_nulos = comportamiento_df.groupby('ID_CLIENTE').filter(lambda x:x['DIAS_VENCIDOS'].isnull().any())
filtro_nulos['ID_CLIENTE'].value_counts().shape[0]

In [ ]:
comportamiento_df['ID_CLIENTE'].value_counts().shape[0]

##### De un total de 45,985 clientes con comportamiento del manejo de tarjeta, 25,005 clientes tienen al menos un campo nulo en dias vencidos.

##### Se consideran todas las observaciones para definir el tipo de pagador toda vez que podemos asegurar un mal comportamiento con información cierta.

##### Score de comportamiento:
Mayor que 25 y menor igual que 30, se puntúa el comportamiento con 0.25.
Mayor que 30, se puntúa con 1.

In [ ]:
# funcion para score comportamiento
def score_comportamiento(dias_vencidos):
    if dias_vencidos > 25 and dias_vencidos <= 30:
        return 0.25
    elif dias_vencidos > 30:
        return 1
    else:
        return 0

In [ ]:
comportamiento_df['SCORE'] = comportamiento_df['DIAS_VENCIDOS'].apply(score_comportamiento)
comportamiento_df['MESES_ANTIGUEDAD'] = (comportamiento_df.groupby('ID_CLIENTE')['MESES'].transform('min')) * (-1)
comportamiento_df

In [ ]:
calificacion_df = comportamiento_df[["ID_CLIENTE","MESES_ANTIGUEDAD","SCORE"]].groupby(["ID_CLIENTE","MESES_ANTIGUEDAD"]).sum().reset_index()
calificacion_df

In [ ]:
# definir etiqueta de mal pagador
def tipo_cliente(meses_antiguedad, score_total):
    if meses_antiguedad <= 6 and score_total > 0:
        return 'mal pagador'
    elif meses_antiguedad > 6 and meses_antiguedad <= 12 and score_total > 0.5:
        return 'mal pagador'
    elif meses_antiguedad > 12 and meses_antiguedad <= 24 and score_total >= 1.5:
        return 'mal pagador'
    elif meses_antiguedad > 24 and meses_antiguedad <= 36 and score_total >= 2.5:
        return 'mal pagador'
    elif meses_antiguedad > 36 and meses_antiguedad <= 48 and score_total >= 3.5:
        return 'mal pagador'
    elif meses_antiguedad > 48 and meses_antiguedad <= 60 and score_total >= 4.5:
        return 'mal pagador'
    elif meses_antiguedad > 60 and score_total >= 5.5:
        return 'mal pagador'
    else:
        return 'buen pagador'

In [ ]:
calificacion_df['TIPO_CLIENTE'] = calificacion_df.apply(lambda x: tipo_cliente(x['MESES_ANTIGUEDAD'], x['SCORE']), axis=1)
calificacion_df['TIPO_CLIENTE'].value_counts(normalize=True)

### Join con información adicional del cliente

In [ ]:
calificacion_df.isnull().sum()

In [ ]:
calificacion_df.info()

No se tiene valores nulos con el nuevo dataset con la definición de mal pagador

In [ ]:
#Se crea un nuevo data con las dos columnas solicitadas CLIENTE_ID (ID_CLIENTE) y mal_pagador (TIPO_CLIENTE)
definicion_df = calificacion_df[["ID_CLIENTE", "TIPO_CLIENTE"]]
definicion_df

In [ ]:
#Análisis del dataset con la información de los clientes
informacion_df.info()

In [ ]:
informacion_df.drop_duplicates()
informacion_df

In [ ]:
informacion_df.isnull().sum()

##### El 31% de las observaciones para la información de las tarjetas es nula en la descripción de la profesión

In [ ]:
id_informacion_df = informacion_df["ID_CLIENTE"].value_counts()
id_informacion_df

In [ ]:
#Observando los valores repetidos
repetido_informacion_df = id_informacion_df[id_informacion_df > 1].index
repetido_informacion_df

In [ ]:
repetido_filtro_df = informacion_df[informacion_df['ID_CLIENTE'].isin(repetido_informacion_df)].sort_values(by="ID_CLIENTE")
repetido_filtro_df

Se descartan los ids que tienen más de una información porque los valores cambiantes no siguen ningún patrón (MCAR)

In [ ]:
#Obteniedo los id con información única
filtered_informacion_df = id_informacion_df[id_informacion_df == 1].index
filtered_informacion_df

In [ ]:
#Filtrando los ids únicos
unicos_informacion_df = informacion_df[informacion_df['ID_CLIENTE'].isin(filtered_informacion_df)]
unicos_informacion_df

94 registros se han descartado de la información de clientes

In [ ]:
unificado_df = unicos_informacion_df.merge(definicion_df, how="left", on="ID_CLIENTE")
unificado_df

In [ ]:
#Analizando la información unificada
unificado_df.info()

Hay 402006 registros lo que representa el 91 % de información nula porque los ids de clientes no estaban en la definición del tipo de cliente.
Es un categoría necesaria para analizar el tipo de cliente se va a trabajar con clientes que si estén categorizados

In [ ]:
unificado_df[unificado_df.duplicated() == True]

No se tiene valores duplicados ya que desde el inicio se usó la función drop_duplicates para no trabajar con valores repetidos

In [ ]:
#Se conservan los registros que no tienen valores nulos para el tipo de cliente según la definición de mal pagador
clientes_df = unificado_df[unificado_df["TIPO_CLIENTE"].isnull() == False]
clientes_df.info()

In [ ]:
clientes_df[["TIPO_CLIENTE", "ID_CLIENTE"]].groupby("TIPO_CLIENTE").count()

### Data quality

El 31% de la información que se ha recolectado del cliente no ha indicado su profesión.

In [ ]:
segun_genero_df = clientes_df[["GENERO", "PROFESION", "ID_CLIENTE"]].groupby("GENERO").count()
segun_genero_df

In [ ]:
segun_genero_df["SIN PROFESION"] = segun_genero_df["ID_CLIENTE"] - segun_genero_df["PROFESION"]
segun_genero_df["% SIN PROFESION"] = (segun_genero_df["SIN PROFESION"] * 100)/segun_genero_df["ID_CLIENTE"]
segun_genero_df

In [ ]:
clientes_df.isnull().sum()/clientes_df.shape[0]


El 36 % de mujeres no ha indicado que profesión tienen y el 21% de los hombres prefirieron no decir a que se dedican

### Feature transformation

In [ ]:
clientes_df.describe()

In [ ]:
fecha_fijada = datetime(2024, 6, 29).date()
clientes_df['FECHA_NACIMIENTO'] = clientes_df['DIAS_DESDE_NACIMIENTO'].apply(lambda x: fecha_fijada + timedelta(days=x))
clientes_df['ANIO_NACIMIENTO'] = clientes_df['FECHA_NACIMIENTO'].apply(lambda x: x.year)
clientes_df

In [ ]:
#funcion para etiqueta generacional
def generacion(anio_nacimiento):
    if anio_nacimiento >= 1997:
        return 'Generación Z'
    elif 1981 <= anio_nacimiento < 1997:
        return 'Millennials'
    elif 1965 <= anio_nacimiento < 1981:
        return 'Generación X'
    elif 1946 <= anio_nacimiento < 1965:
        return 'Baby Boomers'
    else:
        return 'Generación Silenciosa'
clientes_df['GENERACION'] = clientes_df['ANIO_NACIMIENTO'].apply(generacion)

In [ ]:
clientes_df['GENERACION'].value_counts()

In [ ]:
clientes_df.isnull().sum()/clientes_df.shape[0]


### Análisis descriptivo

In [ ]:
generacion_df = pd.crosstab(clientes_df['GENERACION'], clientes_df["TIPO_CLIENTE"], normalize="index")
generacion_df

In [ ]:
generacion_df.reindex(["Generación Z", "Millennials", "Generación X", "Baby Boomers"])

### Análisis correlacional 

In [ ]:
clientes_df

In [ ]:
clientes_df.isnull().sum()/clientes_df.shape[0]


In [ ]:
clientes_df["TIPO_CLIENTE"].value_counts()

In [ ]:
clientes_df.dtypes

In [ ]:
## no aparece la columna TIPO_CLIENTE porque no es un valor numérico 
correlation_matrix=clientes_df.corr(numeric_only=True)
correlation_matrix

In [ ]:
def conversion_tipo_cliente (tipo_cliente):
    if tipo_cliente == "buen pagador": 
        return 1
    else:
        return 0

In [ ]:
clientes_df['NUM_TIPO_CLIENTE'] = clientes_df.apply(lambda x: conversion_tipo_cliente(x["TIPO_CLIENTE"]), axis=1)
correlation_matrix = clientes_df.corr(numeric_only=True)


In [ ]:
#df_encoded = pd.get_dummies(clientes_df)
#correlation_matrix = df_encoded.corr()
#plt.figure(figsize=(10, 8))
#sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')


In [ ]:
## con figsize ajustamos el tamano de los cuadros de la matriz
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", center=0, fmt=".2f")
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
clientes_df["INGRESO_ANUAL"].describe().astype(int)

In [ ]:
clientes_df["NIVEL_EDUCACION"].value_counts()

In [ ]:
## correlacion nivel de educacion vs tipo cliente
educacion_df = pd.crosstab(clientes_df['NIVEL_EDUCACION'], clientes_df["TIPO_CLIENTE"], normalize="index")
educacion_df

In [ ]:
educacion_df.reindex(["Academic degree", "Higher education", "Secondary / secondary special", "Lower secondary", "Incomplete higher"])


In [ ]:
educacion_df.plot(kind='bar', stacked=True)
plt.xlabel('Niveles de Educación')
plt.ylabel('Proporción de Tipos de Cliente')
plt.title('Distribución de Tipos de Cliente por Niveles de Educación')
plt.show()

In [ ]:
clientes_df["INGRESO_ANUAL"].max()

In [ ]:
clientes_df["INGRESO_ANUAL"].min()

In [ ]:
#se tendran tres rangos de ingresos 
resta_ingresos_df = (clientes_df["INGRESO_ANUAL"].max())-(clientes_df["INGRESO_ANUAL"].min())
resta_ingresos_df/3

In [ ]:
def rangos_ingresos(ingresos_anuales):
    if ingresos_anuales <= (27000 + 516000):
        return "Ingresos bajos"
    elif ingresos_anuales >= (1575000 - 516000):
        return "Ingresos altos"
    else:
        return "Ingresos medios"
    
clientes_df["NIVEL_INGRESOS"] = clientes_df["INGRESO_ANUAL"].apply(rangos_ingresos)


In [ ]:
clientes_df

In [ ]:
clientes_df["NIVEL_INGRESOS"].value_counts()

In [ ]:
ingresos_df = pd.crosstab(clientes_df["NIVEL_INGRESOS"], clientes_df["TIPO_CLIENTE"], normalize="index")
ingresos_df

In [ ]:
ingresos_df.reindex(["Ingresos bajos", "Ingresos medios", "Ingresos altos"])

In [ ]:
ingresos_df.plot(kind='bar', stacked=True)
plt.xlabel('Niveles de Ingresos')
plt.ylabel('Proporción de Tipos de Cliente')
plt.title('Distribución de Tipos de Cliente por Niveles de Ingresos')
plt.show()


In [ ]:
union_df = pd.crosstab([clientes_df["NIVEL_INGRESOS"], clientes_df["NIVEL_EDUCACION"]] , clientes_df["TIPO_CLIENTE"], normalize="index")
union_df



In [ ]:
union_df.plot(kind='bar', stacked=True)
plt.xlabel('Niveles de Ingresos')
plt.ylabel('Proporción de Tipos de Cliente')
plt.title('Distribución de Tipos de Cliente por Niveles de Ingresos')
plt.show()